In [39]:
import email, smtplib, datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd

def data_to_send():
    df = pd.read_csv('/Users/Mahmud/Desktop/stock_market_tracker/test.csv').round({'close':2, 'rsi':2, 'macd_diff_signal':2})
    stocks = pd.read_excel('/Users/Mahmud/Desktop/stock_market_tracker/test_track.xlsx')
    merged = pd.merge(df,stocks,how='left',left_on = 'ticker', right_on='Ticker')
    merged = merged[['ticker', 'Company Name', 'Industry', 'date', 'close', 'rsi', 'macd_diff_signal','daily mean volume']]
    merged = merged.rename(columns = {'ticker':'Ticker', 'date':'Date', 'close':'Close', 'rsi':'RSI', 'macd_diff_signal':'MACD'})


    daily_df = merged[(merged['Date'] == merged.Date.max()) & 
                      (merged['RSI']<35) & 
                      (merged['Close']>10) & 
                      (merged['daily mean volume'] > 500000)].sort_values(by=['RSI','MACD']) 
                      

    return daily_df

def email_connection(email, msg, data):

    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()
    s.login('entra.daily@gmail.com', 'Pomidor12')
    msg['From']= email
    msg['To']= 'mahmudkouli@gmail.com'
    #msg['CC']='ksenia.ter@gmail.com'
    msg['Subject']="Entra Alert, " + str(datetime.date.today())

    msg.attach(MIMEText(data.to_html(), 'html'))

    s.send_message(msg)
    print('Successfully sent email')

    s.quit()

def main():
    email = 'entra.daily@gmail.com'
    msg = MIMEMultipart()

    data = data_to_send()[['Ticker', 'Company Name', 'Industry', 'Date', 'Close', 'RSI', 'MACD']]
    week_no = datetime.datetime.today().weekday()

    if week_no < 7:
        email_connection(email, msg, data)
    else:
        pass

if __name__ == "__main__":
    main()
    

Successfully sent email


In [38]:
df[df['ticker']=='BA']

,ch_osc,close,date,high,low,macd,macd_diff_signal,rsi,signal_line,ticker,volume,daily mean volume
45018,0.000000e+00,340.60,2018-07-11,344.3900,339.1900,0.000000,0.00,NaN,0.000000,BA,3056866.0,4883010
45019,1.347834e+05,346.03,2018-07-12,346.6000,342.7374,0.121827,0.05,NaN,0.067682,BA,2030678.0,4883010
45020,1.962468e+05,350.79,2018-07-13,351.2400,346.2000,0.302839,0.14,NaN,0.164058,BA,2281610.0,4883010
45021,2.668035e+05,356.10,2018-07-16,356.3400,352.3500,0.569461,0.27,NaN,0.301389,BA,2889033.0,4883010
45022,5.193801e+04,356.88,2018-07-17,359.0000,355.0400,0.733388,0.30,NaN,0.429899,BA,2730514.0,4883010
45023,1.177980e+05,360.23,2018-07-18,361.8600,354.2638,0.961394,0.39,NaN,0.573964,BA,3090089.0,4883010
45024,-2.934284e+05,355.33,2018-07-19,359.3900,355.2600,0.869078,0.22,NaN,0.648649,BA,2711744.0,4883010
45025,-2.635827e+05,354.90,2018-07-20,356.9900,353.0000,0.772753,0.09,NaN,0.678474,BA,1817836.0,4883010
45026,-2.986563e+05,353.27,2018-07-23,355.0000,352.3900,0.613422,-0.05,NaN,0.663446,BA,1943747.0,4883010
45027,-5.138491e+04,358.27,2018-07-24,359.4700,354.6400,0.749018,0.07,NaN,0.682619,BA,3181965.0,4883010
